# Environement Setup

In [ ]:
# @title 1. Setup Progetto (MistakeDetection)
import sys, os
try:
    from google.colab import drive, userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

REPO_NAME = 'MistakeDetection'

# --- CONFIGURAZIONE PATH ---
if IS_COLAB:
    print("☁️ Colab rilevato.")
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')

    # DEFINIZIONE GLOBALE PROJECT_DIR (Importante per le celle successive!)
    PROJECT_DIR = "/content/drive/MyDrive/MistakeDetection"

    # Fallback se la cartella ha un nome diverso
    if not os.path.exists(PROJECT_DIR):
        if os.path.exists("/content/drive/MyDrive/CaptainCook4D"):
            PROJECT_DIR = "/content/drive/MyDrive/CaptainCook4D"

    print(f"📂 Cartella Progetto su Drive: {PROJECT_DIR}")

    GITHUB_USER = 'MarcoPernoVDP'
    try:
        TOKEN = userdata.get('GITHUB_TOKEN')
        REPO_URL = f'https://{TOKEN}@github.com/{GITHUB_USER}/{REPO_NAME}.git'
    except:
        REPO_URL = f'https://github.com/{GITHUB_USER}/{REPO_NAME}.git'

    ROOT_DIR = f'/content/{REPO_NAME}'

    if not os.path.exists(ROOT_DIR):
        print(f"📥 Clonazione {REPO_NAME}...")
        !git clone {REPO_URL}
    else:
        print(f"🔄 Aggiornamento {REPO_NAME}...")
        %cd {ROOT_DIR}
        !git pull
        %cd /content
else:
    print("Ambiente locale rilevato.")
    ROOT_DIR = os.getcwd()
    while not os.path.exists(os.path.join(ROOT_DIR, '.gitignore')) and ROOT_DIR != os.path.dirname(ROOT_DIR):
        ROOT_DIR = os.path.dirname(ROOT_DIR)
    PROJECT_DIR = ROOT_DIR # In locale coincidono spesso

if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)

print(f"✅ ROOT_DIR impostata a: {ROOT_DIR}")

In [1]:
# @title 2. Setup ActionFormer
import os
import sys
import shutil
import subprocess

# Usiamo un workspace separato
AF_WORKDIR = "/content/actionformer_workspace"
os.makedirs(AF_WORKDIR, exist_ok=True)
os.chdir(AF_WORKDIR)

REPO_NAME = "multi_step_localization"
AF_REPO_PATH = os.path.join(AF_WORKDIR, REPO_NAME)

# 1. Clone
if not os.path.exists(AF_REPO_PATH):
    print("📥 Clonazione ActionFormer (con --recursive)...")
    try:
        subprocess.run(["git", "clone", "--recursive", "https://github.com/CaptainCook4D/multi_step_localization.git"], check=True)
    except Exception as e:
        print(f"⚠️ Clone recursive fallito, provo standard...")
        subprocess.run(["git", "clone", "https://github.com/CaptainCook4D/multi_step_localization.git"], check=True)

os.chdir(AF_REPO_PATH)

# 2. Fix Path Libs
if os.path.exists(os.path.join(AF_REPO_PATH, "actionformer", "libs", "utils")):
    UTILS_PATH = os.path.join(AF_REPO_PATH, "actionformer", "libs", "utils")
    PATCH_DIR = os.path.join(AF_REPO_PATH, "actionformer")
elif os.path.exists(os.path.join(AF_REPO_PATH, "libs", "utils")):
    UTILS_PATH = os.path.join(AF_REPO_PATH, "libs", "utils")
    PATCH_DIR = AF_REPO_PATH
else:
    # Tentativo update submodule
    print("⚠️ Cartella libs non trovata, provo update submodule...")
    subprocess.run(["git", "submodule", "update", "--init", "--recursive"], check=True)
    if os.path.exists(os.path.join(AF_REPO_PATH, "libs", "utils")):
        UTILS_PATH = os.path.join(AF_REPO_PATH, "libs", "utils")
        PATCH_DIR = AF_REPO_PATH
    else:
        raise FileNotFoundError("CRITICO: Impossibile trovare libs/utils.")

print(f"✅ Cartella Utils: {UTILS_PATH}")

# 3. Installazione & Patch
print("📦 Installazione dipendenze...")
subprocess.run([sys.executable, "-m", "pip", "install", "pyyaml", "scipy"], check=True)

print("🩹 Patch NumPy 2.0...")
with open(os.path.join(PATCH_DIR, "numpy_patch.py"), "w") as f:
    f.write("import numpy as np\n")
    f.write("try:\n  if not hasattr(np, 'float'): np.float = np.float64\nexcept: pass\n")
    f.write("try:\n  if not hasattr(np, 'int'): np.int = np.int_\nexcept: pass\n")

# Inietta patch in eval.py
eval_path = os.path.join(AF_REPO_PATH, "eval.py")
if os.path.exists(eval_path):
    with open(eval_path, "r") as f: content = f.read()
    if "import numpy_patch" not in content:
        with open(eval_path, "w") as f:
            f.write("import sys\nsys.path.append('actionformer')\nimport numpy_patch\n" + content)

# 4. Compilazione
print("⚙️ Compilazione CUDA...")
os.chdir(UTILS_PATH)
subprocess.run([sys.executable, "setup.py", "install"], check=True)

os.chdir(AF_REPO_PATH)
print("\n✅ Ambiente ActionFormer pronto.")

Mounted at /content/drive
📂 Cartella Progetto rilevata: /content/drive/MyDrive/MistakeDetection
/content/actionformer_workspace
📥 Clonazione ActionFormer...
Cloning into 'multi_step_localization'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 79 (delta 26), reused 66 (delta 20), pack-reused 0 (from 0)
Receiving objects: 100% (79/79), 1.45 MiB | 5.09 MiB/s, done.
Resolving deltas: 100% (26/26), done.
/content/actionformer_workspace/multi_step_localization
📦 Installazione librerie...
ERROR: Could not find a version that satisfies the requirement torch==1.11.0+cu113 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0, 2.9.0, 2.9.1)
ERROR: No matching distribution found for torch==1.11.0+cu113
⚙️ Compilazione kernel CUDA...
[Errno 2] No such file or directory: './libs/utils'
/content/actionformer_workspace/multi_step_localization
pyth

In [ ]:
# @title 3. Estrazione Feature Omnivore
import zipfile
import shutil
import os
from tqdm import tqdm

# --- CONFIGURAZIONE VARIABILI (Self-Contained) ---
if 'PROJECT_DIR' not in locals():
    # Tenta di indovinare il path
    if os.path.exists("/content/drive/MyDrive/MistakeDetection"):
        PROJECT_DIR = "/content/drive/MyDrive/MistakeDetection"
    elif os.path.exists("/content/drive/MyDrive/CaptainCook4D"):
        PROJECT_DIR = "/content/drive/MyDrive/CaptainCook4D"
    else:
        # Fallback locale se non trova nulla (o se sei in locale)
        PROJECT_DIR = os.getcwd()

if 'ROOT_DIR' not in locals():
    # Tenta di trovare il repo clonato
    possible_roots = [
        os.path.join(PROJECT_DIR, "MistakeDetection"),
        "/content/MistakeDetection",
        PROJECT_DIR
    ]
    for r in possible_roots:
        if os.path.exists(os.path.join(r, ".git")):
            ROOT_DIR = r
            break
    if 'ROOT_DIR' not in locals(): ROOT_DIR = PROJECT_DIR

# --- RICERCA ZIP ---
POSSIBLE_PATHS = [
    os.path.join(PROJECT_DIR, "_file", "omnivore.zip"),
    os.path.join(PROJECT_DIR, "data", "omnivore.zip"),
    os.path.join(PROJECT_DIR, "omnivore.zip"),
    # Path specifici colab
    "/content/drive/MyDrive/MistakeDetection/omnivore.zip",
    "/content/drive/MyDrive/MistakeDetection/data/omnivore.zip"
]

ZIP_PATH = None
for p in POSSIBLE_PATHS:
    if os.path.exists(p):
        ZIP_PATH = p
        break

LOCAL_FEAT_DIR = "/content/temp_omnivore_features"

if ZIP_PATH is None:
    print(f"❌ ERRORE: Non trovo 'omnivore.zip'.")
    print(f"   Ho cercato in: {POSSIBLE_PATHS}")
else:
    print(f"📂 Trovato Zip: {ZIP_PATH}")
    print(f"⏳ Estrazione in: {LOCAL_FEAT_DIR}...")

    if os.path.exists(LOCAL_FEAT_DIR):
        try:
            shutil.rmtree(LOCAL_FEAT_DIR)
        except: pass # Ignora errori permessi
    os.makedirs(LOCAL_FEAT_DIR, exist_ok=True)

    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(LOCAL_FEAT_DIR)

    print(f"✅ Estrazione completata.")

🔧 Workdir: c:\Users\enric\Desktop\MistakeDetection\temp_af_build
📥 Clonazione ActionFormer...
🔎 Ricerca file setup.py...
❌ CRITICO: setup.py non trovato! Elenco contenuto scaricato:
multi_step_localization/
    .git/
        hooks/
        info/
        logs/
            refs/
                heads/
                remotes/
                    origin/
        objects/
            info/
            pack/
        refs/
            heads/
            remotes/
                origin/
            tags/
    actionformer/
    captaincook/
    captaincook_actionformer_annotations/
        combined/
        normal/
    metrics_data/
    model_outputs/
🔄 Directory ripristinata.


FileNotFoundError: Struttura repository non valida

# Features Extraction

In [ ]:
# @title 4. Inferenza ActionFormer
import yaml
import os
import subprocess
import glob

# --- CONFIGURAZIONE VARIABILI (Self-Contained) ---
AF_WORKDIR = "/content/actionformer_workspace"
# Cerca la repo ActionFormer (potrebbe essere in libs o root)
if os.path.exists(os.path.join(AF_WORKDIR, "multi_step_localization")):
    AF_REPO_PATH = os.path.join(AF_WORKDIR, "multi_step_localization")
else:
    AF_REPO_PATH = AF_WORKDIR # Fallback

if 'PROJECT_DIR' not in locals():
    if os.path.exists("/content/drive/MyDrive/MistakeDetection"):
        PROJECT_DIR = "/content/drive/MyDrive/MistakeDetection"
    else:
        PROJECT_DIR = "/content/drive/MyDrive/CaptainCook4D" # Fallback

LOCAL_FEAT_DIR = "/content/temp_omnivore_features"

# --- 1. GESTIONE ANNOTAZIONI ---
# Cerchiamo se hai già il json pronto o se dobbiamo convertirlo
ANNOTATION_JSON_TARGET = os.path.join(AF_WORKDIR, "actionformer_split.json")
USER_ANNOTATION_DIR = os.path.join(PROJECT_DIR, "annotation_json")

if not os.path.exists(ANNOTATION_JSON_TARGET):
    print("⚠️ 'actionformer_split.json' non trovato. Controllo cartella annotazioni...")
    if os.path.exists(USER_ANNOTATION_DIR):
        print("🚀 Avvio conversione automatica...")
        converter = os.path.join(AF_REPO_PATH, "convert_to_action_former_json.py")
        if os.path.exists(converter):
            try:
                subprocess.run(["python", converter,
                               "--annotation_folder", USER_ANNOTATION_DIR,
                               "--output_file", ANNOTATION_JSON_TARGET], check=True, cwd=AF_REPO_PATH)
                print("✅ Conversione riuscita!")
            except Exception as e:
                print(f"❌ Errore conversione: {e}")
        else:
            print("❌ Script convertitore non trovato!")

# --- 2. RICERCA DISPERATA DEL CHECKPOINT ---
CKPT_NAME = "actionformer_best.pth"
MODEL_CKPT = None

print(f"🔍 Ricerca {CKPT_NAME} in corso...")

# Lista di posti dove cercare
search_locations = [
    os.path.join(PROJECT_DIR, "checkpoints"),
    PROJECT_DIR,
    # Cerca anche dentro il repo scaricato (magari è in model_outputs o pretrained)
    os.path.join(AF_REPO_PATH, "model_outputs"),
    os.path.join(AF_REPO_PATH, "checkpoints"),
    os.path.join(AF_REPO_PATH, "pretrained"),
    AF_REPO_PATH
]

# 1. Cerca nei path specifici
for loc in search_locations:
    path = os.path.join(loc, CKPT_NAME)
    if os.path.exists(path):
        MODEL_CKPT = path
        break

# 2. Se non trovato, cerca ricorsivamente QUALSIASI .pth nel repo scaricato
if not MODEL_CKPT:
    print("⚠️ Checkpoint non trovato nei percorsi standard. Scansione repo...")
    pth_files = glob.glob(os.path.join(AF_REPO_PATH, "**", "*.pth"), recursive=True)
    if pth_files:
        # Prendi il file più grande (probabilmente è il modello)
        best_candidate = max(pth_files, key=os.path.getsize)
        print(f"   -> Trovato possibile candidato: {best_candidate}")
        MODEL_CKPT = best_candidate

if not MODEL_CKPT:
    print("🛑 ERRORE BLOCCANTE: Il file dei pesi non c'è.")
    print("   ActionFormer NON è incluso nel repo GitHub (è troppo grande).")
    print(f"   Devi scaricarlo dai link forniti nel corso e caricarlo qui: {os.path.join(PROJECT_DIR, 'checkpoints')}")
    # Creiamo la cartella per facilitare l'upload all'utente
    os.makedirs(os.path.join(PROJECT_DIR, "checkpoints"), exist_ok=True)
    raise FileNotFoundError(f"Manca {CKPT_NAME}")

print(f"✅ Modello trovato: {MODEL_CKPT}")

# --- 3. ESECUZIONE ---
OUTPUT_INFERENCE = os.path.join(AF_WORKDIR, "output_results")
os.makedirs(OUTPUT_INFERENCE, exist_ok=True)

config_data = {
    'dataset': {
        'json_file': ANNOTATION_JSON_TARGET,
        'feat_folder': LOCAL_FEAT_DIR,
        'file_prefix': '',
        'file_ext': '.npz',
        'input_dim': 1024,
        'feat_stride': 16,
        'num_classes': 24,
        'default_fps': 30,
    },
    'eval': {
        'batch_size': 1,
        'nms_score_thres': 0.1,
    },
    'loader': {'batch_size': 1, 'num_workers': 2},
    'model': {
        'backbone_arch': [2, 2, 5],
        'regression_range': [[0, 4], [4, 8], [8, 16], [16, 32], [32, 10000]],
        'fpn_dim': 256, 'head_dim': 512, 'use_abs_pe': False, 'max_buffer_len_factor': 6.0, 'n_mha_win_size': 19
    },
    'train': {'head_dim': 512}
}

os.makedirs(os.path.join(AF_REPO_PATH, "configs"), exist_ok=True)
CONFIG_PATH = os.path.join(AF_REPO_PATH, "configs", "run_config.yaml")
with open(CONFIG_PATH, 'w') as f:
    yaml.dump(config_data, f)

print("🚀 Avvio Inferenza...")
os.chdir(AF_REPO_PATH)
!python eval.py "{CONFIG_PATH}" "{MODEL_CKPT}" --save_results --output_folder "{OUTPUT_INFERENCE}"

# Pooling, Zipping, Upload

In [ ]:
# @title 5. Pooling e Export Finale
import numpy as np
import json
import glob
import shutil

if 'PROJECT_DIR' not in locals(): PROJECT_DIR = "/content/drive/MyDrive/MistakeDetection"

RESULTS_JSON = os.path.join(AF_WORKDIR, "output_results", "results.json")
TEMP_EMB_DIR = os.path.join(AF_WORKDIR, "step_embeddings_temp")
if os.path.exists(TEMP_EMB_DIR): shutil.rmtree(TEMP_EMB_DIR)
os.makedirs(TEMP_EMB_DIR, exist_ok=True)

FINAL_DATA_DIR = os.path.join(PROJECT_DIR, "data")
ZIP_OUTPUT_NAME = "step_embeddings"

def load_npz(path):
    try:
        with np.load(path) as data:
            if 'features' in data: return data['features']
            if 'arr_0' in data: return data['arr_0']
            return data[list(data.keys())[0]]
    except: return None

if os.path.exists(RESULTS_JSON):
    print("🚀 Inizio elaborazione embedding...")

    with open(RESULTS_JSON, 'r') as f:
        data = json.load(f)
        preds = data.get('results', data)

    # Mappa file locali
    file_map = {os.path.basename(f).split('.')[0]: f for f in glob.glob(os.path.join(LOCAL_FEAT_DIR, "*.npz"))}

    count = 0
    fps, stride = 30.0, 16.0
    feat_rate = fps / stride

    for video_id, segments in preds.items():
        feat_path = file_map.get(video_id)
        if not feat_path:
            candidates = [f for k, f in file_map.items() if video_id in k]
            if candidates: feat_path = candidates[0]

        if not feat_path: continue

        full_feats = load_npz(feat_path)
        if full_feats is None: continue

        step_embeddings = []
        step_metadata = []

        for seg in segments:
            t_start, t_end = seg['segment']
            if seg['score'] < 0.1: continue

            s_idx, e_idx = int(t_start * feat_rate), int(t_end * feat_rate)
            s_idx, e_idx = max(0, s_idx), min(len(full_feats), e_idx)
            if e_idx <= s_idx: e_idx = s_idx + 1

            if s_idx < len(full_feats):
                pooled = np.mean(full_feats[s_idx:e_idx], axis=0)
                step_embeddings.append(pooled)
                step_metadata.append(seg)

        if step_embeddings:
            np.savez_compressed(
                os.path.join(TEMP_EMB_DIR, f"{video_id}_steps.npz"),
                embeddings=np.array(step_embeddings),
                metadata=step_metadata
            )
            count += 1

    print(f"\n✅ Generati {count} file .npz")

    # Zipping e Copy su Drive
    print(f"📦 Creazione archivio ZIP...")
    zip_path = shutil.make_archive(
        base_name=os.path.join(AF_WORKDIR, ZIP_OUTPUT_NAME),
        format='zip',
        root_dir=TEMP_EMB_DIR
    )

    print(f"☁️ Upload su Drive: {FINAL_DATA_DIR}...")
    try:
        shutil.copy2(zip_path, FINAL_DATA_DIR)
        print(f"✅ SUCCESSO! File salvato in: {os.path.join(FINAL_DATA_DIR, ZIP_OUTPUT_NAME + '.zip')}")
    except Exception as e:
        print(f"❌ Errore durante l'upload: {e}")

else:
    print("❌ Nessun risultato inferenza trovato. (Verifica Cella 4)")